---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [159]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from operator import attrgetter
from itertools import product

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [49]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    statesRegions = []
    for line in open('university_towns.txt','r').readlines():
        if '[edit]' in line:
            state = line.split("[")[0]
        else:
            regionName = line.split()[0]
            statesRegions.append([state,regionName])
    return pd.DataFrame(statesRegions, columns=['State', 'Region'])
get_list_of_university_towns()

,State,Region
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [32]:
def searching_for_end_recession(i,start,qtlyGPD):
    rangeRecession, seaching_end_recession = [], True
    while(seaching_end_recession):
        t_1,t_2,t_3 = qtlyGPD.loc[i], qtlyGPD.loc[i+1], qtlyGPD.loc[i+2]
        if t_1.gpd < t_2.gpd < t_3.gpd:
            end, seaching_end_recession = t_3.quarterly, False
            rangeRecession+=[t_1,t_2,t_3]
            bottom = min(rangeRecession,key=attrgetter('gpd')).quarterly
            return i, [start,end,bottom]
        else:
            rangeRecession+=[t_1,t_2,t_3]
            i+=1
                
def get_recession_structure():
    path = r'~/Downloads/quarterlyGPD.csv'
    qtlyGPD = pd.read_csv(path)
    i,endSerie,recessions = 0,len(qtlyGPD)-2,[]
    while i < endSerie:
        t_1,t_2,t_3 = qtlyGPD.loc[i], qtlyGPD.loc[i+1], qtlyGPD.loc[i+2]
        if t_1.gpd > t_2.gpd > t_3.gpd:
            i,recession = searching_for_end_recession(i,t_1.quarterly,qtlyGPD)
            recessions.append(recession)
        i+=1      
    return pd.DataFrame(recessions, columns=['Start','End','Bottom'])
recessions = get_recession_structure()

In [39]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    return recessions.Start                 
get_recession_start()

0    1948q4
1    1953q2
2    1957q3
3    2008q3
Name: Start, dtype: object

In [40]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    return recessions.End 
get_recession_end()

0    1950q2
1    1954q3
2    1958q3
3    2009q4
Name: End, dtype: object

In [41]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    return recessions.Bottom
get_recession_bottom()

0    1949q4
1    1954q1
2    1958q1
3    2009q2
Name: Bottom, dtype: object

In [187]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    ct = pd.read_csv("City_Zhvi_AllHomes.csv", index_col=['State', 'RegionName'],usecols=[1,2]+range(51,252,1))
    newCols = [str(i)+j for i,j in product(range(2000,2017), ['q1', 'q2', 'q3','q4'])][:-1]
    for col in newCols:
        ct[col] = ct[ct.columns[0:3]].mean(axis=1)
        ct = ct.drop(ct.columns[0:3], axis=1)
    return ct
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.162000e+05,5.190333e+05,5.218000e+05,5.233333e+05,5.321000e+05,5.477667e+05,5.617000e+05,5.728333e+05,5.854000e+05,6.022667e+05
CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.984333e+05,5.089333e+05,5.179667e+05,5.270333e+05,5.354667e+05,5.449333e+05,5.568667e+05,5.682667e+05,5.810000e+05,5.895333e+05
IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.881667e+05,1.902000e+05,1.936000e+05,1.949333e+05,1.989000e+05,2.014333e+05,2.027333e+05,2.040000e+05,2.058667e+05,2.079667e+05
PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.146000e+05,1.159667e+05,1.166000e+05,1.167333e+05,1.178000e+05,1.205333e+05,1.221333e+05,1.238667e+05,1.261000e+05,1.305000e+05
AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.662667e+05,1.674667e+05,1.705667e+05,1.733333e+05,1.764333e+05,1.804667e+05,1.851667e+05,1.890000e+05,1.931000e+05,1.983000e+05
NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.690333e+05,1.716000e+05,1.746667e+05,1.776667e+05,1.803333e+05,1.852333e+05,1.887667e+05,1.922000e+05,1.961000e+05,2.025667e+05
CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.747000e+05,4.809333e+05,4.861000e+05,4.933000e+05,5.037667e+05,5.127000e+05,5.208333e+05,5.275333e+05,5.324000e+05,5.423333e+05
TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.054333e+05,1.071333e+05,1.098000e+05,1.125333e+05,1.201333e+05,1.279667e+05,1.343333e+05,1.399333e+05,1.455000e+05,1.553667e+05
CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.818667e+05,7.005333e+05,7.179667e+05,7.374667e+05,7.644000e+05,7.874667e+05,8.053000e+05,8.241667e+05,8.390667e+05,8.349667e+05


In [81]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    rec_st = get_recession_start()
    rec_bot = get_recession_bottom()
    print (rec_st, rec_bot)
    date_range = [a for a in pd.date_range(rec_st, rec_bot, freq='Q')]
    
    uni_t = get_list_of_university_towns()
    housing = convert_housing_data_to_quarters2()
    
    
    out = pd.merge(uni_t, housing, how='inner', left_on=['State', 'RegionName'], right_index=True)
    
    return True 
run_ttest()

2008q4 2009q2


True